<a href="https://colab.research.google.com/github/serinss/seSAC_ML_DL_Class/blob/main/05_3_%ED%8A%B8%EB%A6%AC%EC%9D%98_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%run my_init.py

Mounted at /content/drive


In [2]:
%run my_init.py

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#정형 데이터와 비정형 데이터
- Voting
- bagging(=bootstrap aggregation)
- boosting

#앙상블 학습 - Voting
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer
https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29


In [3]:
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import load_breast_cancer

In [5]:
obj = load_breast_cancer()
type(obj)  # bunch 객체로 리턴 (딕셔너리와 비슷하다)

sklearn.utils.Bunch

In [20]:
print(f"feature_names: {obj.feature_names.shape} = Dimensionality\n{obj.feature_names}")
print("============================================================================\n")
print(f"data: {obj.data.shape} \n{obj.data}")
print("============================================================================\n")
print(f"target_names: {obj.target_names.shape}\n{obj.target_names}")
print("============================================================================\n")
print(f"target: {obj.target.shape}\n{obj.target}")

# 해당 데이터로 Voting해보기 위함임

feature_names: (30,) = Dimensionality
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']

data: (569, 30) 
[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.8

In [23]:
X, y = load_breast_cancer(return_X_y=True)
print(f"X: {X.shape}\n{X}, \n\ny: {y.shape}\n{y}")

# X,y를 리턴해서 쓰면 데이터만 가져오는 것이므로 따로 뽑아서 쓰기 힘들다

X: (569, 30)
[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]], 

y: (569,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
X_train.shape, y_train.shape

((455, 30), (455,))

In [32]:
X_test.shape, y_test.shape

((114, 30), (114,))

In [33]:
lr = LogisticRegression()
knn = KNeighborsClassifier()
vt = VotingClassifier([('LR', lr), {'KNN', knn}])

In [36]:
for clf in [lr, knn, vt]:
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(f"{clf.__class__.__name__}: {accuracy_score(y_test, y_pred):.4f}")
  # 파이선 모든 객체는 __class__ 가 모두 존재함 -> 클래스에 대한 참조를 보면 이름이 존재

# Voting클래스가 제일 좋음 - 기본값으로 진행한 것임
# 일할 때, 앙상블을 안쓸래야 안쓸 수 없음

LogisticRegression: 0.9649
KNeighborsClassifier: 0.9561
VotingClassifier: 0.9737
